# Problem Set 1 (PS1): Deeper Dive on Malkiel's Theorem 4
Malkiel proposed five theorems that govern the price of fixed-income debt securities, e.g., Treasury Bills, Notes, and Bonds, as a function of the duration, yield, and coupon (interest) rate  values:

> [Malkiel, B. G. (1962). Expectations, Bond Prices, and the Term Structure of Interest Rates. The Quarterly Journal of Economics, 76(2), 197–218. https://doi.org/10.2307/1880816](https://www.jstor.org/stable/1880816)

We showed (by simulation) in class that we could observe the essential stipulation of Theorem 4:
* `Theorem 4`: Price movements resulting from equal absolute increases and decreases in yield are asymmetric; i.e., decreasing yields raise bond prices more than the same increase in yields lowers prices.

However, during the discussion of the simulation results, we ran up against an interesting question: 
* __Question__: Would you expect the price asymmetry to increase or decrease with the duration of the note or bond?

### Learning objectives and tasks
Fill me in

## Setup
Set up the computational environment using [the `Include.jl` file](Include.jl). The [`Include.jl` file](Include.jl) loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

## Prerequisite: Load the Treasury auction dataset
We'll explore `T-bill,` `T-note,` and `T-bond` prices from United States Treasury auctions between March 2020 and September 2024 downloaded as a `CSV` file using the [Auction query functionality of TreasuryDirect.gov](https://www.treasurydirect.gov/auctions/auction-query/). We load the `CSV` dataset using the local [`MyTreasuryBillsNotesAndBondsDataSet()` function](src/Files.jl), which returns the auction data as [the `dataset::DataFrame`](https://dataframes.juliadata.org/stable/) variable.

In [11]:
dataset = MyTreasuryBillsNotesAndBondsDataSet()

1819×11 DataFrame
  Row │ CUSIP      Security Type  Security Term  Auction Date  Issue Date  Mat ⋯
      │ String15   String7        String7        String15      String15    Str ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ 912797LS4  Bill           4-Week         09/05/2024    09/10/2024  10/ ⋯
    2 │ 912797MA2  Bill           8-Week         09/05/2024    09/10/2024  11/
    3 │ 912797MW4  Bill           17-Week        09/04/2024    09/10/2024  01/
    4 │ 912797KU0  Bill           42-Day         09/03/2024    09/05/2024  10/
    5 │ 912797LF2  Bill           13-Week        09/03/2024    09/05/2024  12/ ⋯
    6 │ 912797MM6  Bill           26-Week        09/03/2024    09/05/2024  03/
    7 │ 912797MH7  Bill           52-Week        09/03/2024    09/05/2024  09/
    8 │ 912797LK1  Bill           4-Week         08/29/2024    09/03/2024  10/
    9 │ 912797LV7  Bill           8-Week         08/29/2024    09/03/2024  10/ ⋯
   10 │ 91282CLJ8  Note           7-Year         08/29/2024    09/03/2024  08/
   11 │ 912797MR5  Bill           17-Week        08/28/2024    09/03/2024  12/
  ⋮   │     ⋮            ⋮              ⋮             ⋮            ⋮           ⋱
 1810 │ 912828ZG8  Note           2-Year         03/24/2020    03/31/2020  03/
 1811 │ 9127962F5  Bill           52-Week        03/24/2020    03/26/2020  03/ ⋯
 1812 │ 912796UA5  Bill           13-Week        03/23/2020    03/26/2020  06/
 1813 │ 9127962H1  Bill           26-Week        03/23/2020    03/26/2020  09/
 1814 │ 9127962C2  Bill           4-Week         03/19/2020    03/24/2020  04/
 1815 │ 9127962M0  Bill           8-Week         03/19/2020    03/24/2020  05/ ⋯
 1816 │ 912796TZ2  Bill           79-Day         03/19/2020    03/24/2020  06/
 1817 │ 912796SV2  Bill           13-Week        03/16/2020    03/19/2020  06/
 1818 │ 9127962G3  Bill           26-Week        03/16/2020    03/19/2020  09/
 1819 │ 9127962B4  Bill           4-Week         03/12/2020    03/17/2020  04/ ⋯
                                                 6 columns and 1798 rows omitted

Fill me in

In [14]:
bills_dataset = filter(Symbol("Security Type") => x-> x == "Bill", dataset)

1510×11 DataFrame
  Row │ CUSIP      Security Type  Security Term  Auction Date  Issue Date  Mat ⋯
      │ String15   String7        String7        String15      String15    Str ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ 912797LS4  Bill           4-Week         09/05/2024    09/10/2024  10/ ⋯
    2 │ 912797MA2  Bill           8-Week         09/05/2024    09/10/2024  11/
    3 │ 912797MW4  Bill           17-Week        09/04/2024    09/10/2024  01/
    4 │ 912797KU0  Bill           42-Day         09/03/2024    09/05/2024  10/
    5 │ 912797LF2  Bill           13-Week        09/03/2024    09/05/2024  12/ ⋯
    6 │ 912797MM6  Bill           26-Week        09/03/2024    09/05/2024  03/
    7 │ 912797MH7  Bill           52-Week        09/03/2024    09/05/2024  09/
    8 │ 912797LK1  Bill           4-Week         08/29/2024    09/03/2024  10/
    9 │ 912797LV7  Bill           8-Week         08/29/2024    09/03/2024  10/ ⋯
   10 │ 912797MR5  Bill           17-Week        08/28/2024    09/03/2024  12/
   11 │ 912797KT3  Bill           42-Day         08/27/2024    08/29/2024  10/
  ⋮   │     ⋮            ⋮              ⋮             ⋮            ⋮           ⋱
 1501 │ 9127962N8  Bill           8-Week         03/26/2020    03/31/2020  05/
 1502 │ 9127962F5  Bill           52-Week        03/24/2020    03/26/2020  03/ ⋯
 1503 │ 912796UA5  Bill           13-Week        03/23/2020    03/26/2020  06/
 1504 │ 9127962H1  Bill           26-Week        03/23/2020    03/26/2020  09/
 1505 │ 9127962C2  Bill           4-Week         03/19/2020    03/24/2020  04/
 1506 │ 9127962M0  Bill           8-Week         03/19/2020    03/24/2020  05/ ⋯
 1507 │ 912796TZ2  Bill           79-Day         03/19/2020    03/24/2020  06/
 1508 │ 912796SV2  Bill           13-Week        03/16/2020    03/19/2020  06/
 1509 │ 9127962G3  Bill           26-Week        03/16/2020    03/19/2020  09/
 1510 │ 9127962B4  Bill           4-Week         03/12/2020    03/17/2020  04/ ⋯
                                                 6 columns and 1489 rows omitted

Fill me in

In [18]:
notes_bonds_dataset = filter(Symbol("Security Type") => x-> (x == "Note" || "Bond"), dataset)

309×11 DataFrame
 Row │ CUSIP      Security Type  Security Term  Auction Date  Issue Date  Matu ⋯
     │ String15   String7        String7        String15      String15    Stri ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 91282CLJ8  Note           7-Year         08/29/2024    09/03/2024  08/3 ⋯
   2 │ 91282CLK5  Note           5-Year         08/28/2024    09/03/2024  08/3
   3 │ 91282CLH2  Note           2-Year         08/27/2024    09/03/2024  08/3
   4 │ 912810UD8  Bond           20-Year        08/21/2024    09/03/2024  08/1
   5 │ 912810UC0  Bond           30-Year        08/08/2024    08/15/2024  08/1 ⋯
   6 │ 91282CLF6  Note           10-Year        08/07/2024    08/15/2024  08/1
   7 │ 91282CLG4  Note           3-Year         08/06/2024    08/15/2024  08/1
   8 │ 91282CLD1  Note           7-Year         07/25/2024    07/31/2024  07/3
   9 │ 91282CLA7  Note           2-Year         07/24/2024    07/31/2024  07/3 ⋯
  10 │ 91282CLC3  Note           5-Year         07/24/2024    07/31/2024  07/3
  11 │ 91282CLB5  Note           2-Year         07/23/2024    07/31/2024  07/3
  ⋮  │     ⋮            ⋮              ⋮             ⋮            ⋮            ⋱
 300 │ 912828ZP8  Note           3-Year         05/11/2020    05/15/2020  05/1
 301 │ 912828ZK9  Note           2-Year         04/28/2020    04/30/2020  04/3 ⋯
 302 │ 912828ZN3  Note           7-Year         04/28/2020    04/30/2020  04/3
 303 │ 912828ZM5  Note           2-Year         04/27/2020    04/30/2020  04/3
 304 │ 912828ZL7  Note           5-Year         04/27/2020    04/30/2020  04/3
 305 │ 912828ZJ2  Note           5-Year         04/23/2020    04/30/2020  04/1 ⋯
 306 │ 912828ZH6  Note           3-Year         04/06/2020    04/15/2020  04/1
 307 │ 912828ZE3  Note           7-Year         03/26/2020    03/31/2020  03/3
 308 │ 912828ZF0  Note           5-Year         03/25/2020    03/31/2020  03/3
 309 │ 912828ZG8  Note           2-Year         03/24/2020    03/31/2020  03/3 ⋯
                                                  6 columns and 288 rows omitted

Fill me in

In [29]:
discount_model = DiscreteCompoundingModel();

## Task 1: Compute prices for a Bill, Note, and Bond to check our implementation
To verify our code installation (and implementation), the teaching team has selected a random bill, note, and bond from the dataset. In this task, you will construct the corresponding treasury model instance, compute the price for each of these securities, and compare the calculated and observed prices. 

### Check: Compute the price of a treasury bill
Let's compute the price of a `26-Week` T-bill with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04865.` The observed price for this security is $V_{B}$ = `97.6315 USD`. Assume two compounding periods per year.
* Create an instance of [the `MyUSTreasuryZeroCouponBondModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryZeroCouponBondModel) using a [custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), assuming the data above. Pass the data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), and use the short-cut syntax to compute the price. Save the updated model in the `zero_coupon_model` variable. 

In [125]:
zero_coupon_model = build(MyUSTreasuryZeroCouponBondModel, (
    n = 2, par = 100, 
        T = "26-Week" |> securityterm,
        rate = 0.04865,
)) |> discount_model;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [94]:
observed_bill_price = 97.6315;
@assert isapprox(observed_bill_price, zero_coupon_model.price, rtol=1e-4)

### Check: Compute the price of a treasury note
Let's compute the price of a `7-Year` T-Note with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04162` and coupon rate $\bar{c}$ = `0.04125.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.
* Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple})., assuming the data above. Pass in the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `note_coupon_model` variable. 

In [120]:
note_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
    λ = 2, par = 100.0,
        
    T = "7-Year" |> securityterm,
    rate = 0.04162,
    coupon = 0.04125,
    
)) |> discount_model;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [102]:
observed_note_price = 99.7773;
@assert isapprox(observed_note_price, note_coupon_model.price, rtol=1e-4)

### Check: Compute the price of a treasury bond
Let's compute the price of a `30-Year` T-Bond with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04314` and coupon rate $\bar{c}$ = `0.0425.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.
* Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple})., assuming the data above. Pass in the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `bond_coupon_model` variable. 

In [113]:
bond_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
    λ = 2, par = 100.0,
        
    T = "30-Year" |> securityterm,
    rate = 0.04314,
    coupon = 0.0425,
    
)) |> discount_model;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [117]:
observed_bond_price = 98.9288;
@assert isapprox(observed_bond_price, bond_coupon_model.price, rtol=1e-4)

## Task 2: Do bills show the price asymmetry property? 
In class, we showed that notes (and, by extension, bonds) show price change asymmetry. However, we have not explored whether Bills also have this property. In this task, you'll use the treasury bill model that we created above and explore the price asymmetry question by simulation.

To simulate the asymmetry of changes in price following changes in the yield (discount rate), all other values held constant generate a new rate of the form $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value; if $\beta<1$ the perturbed interest rate is _less than_ the nominal rate, if $\beta=1$ the perturbed interest rate is _equals_ the nominal rate, and if $\beta>1$ the perturbed interest rate is _greater than_ the nominal rate. Let's use the `zero_coupon_model` instance from above for this experiment.

* First, specify the number of perturbation values in the `number_of_samples_task_2` variable; for this simulation, let `number_of_samples_task_2 = 7` (odd, so we capture the nominal case as the center data point).
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, compute the perturbation array (stored in the `β::Array{Float64,1}` variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array

In [136]:
number_of_samples_task_2  = 7;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁, stop = β₂, length = number_of_samples_task_2) |> collect;

Your job is to complete the implementation of the `Theorem 4` simulation for Bills, which we started below, and analyze the simulation results. Display the results in a table using the `pretty_table(...)` function exported from the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl)
*  `Hint`: check out [the Julia `deepcopy(...)` function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy). When you complete your implementation, you should see a table similar (numbers different, but similar structure) to the in-class example. Do you see that?

In [157]:
let
    VB = observed_bill_price;
    simulation_results_task_2_array = Array{Float64,2}(undef, number_of_samples_task_2, 3);
    for i ∈ eachindex(β)
        β_value = β[i]
    
        # create a copy of the test_node instance
        model = deepcopy(zero_coupon_model);
        
        ### BEGIN SOLUTION
        model.rate = β_value*zero_coupon_model.rate
        ### END SOLUTION
        
        # compute: use short-cut syntax and compute the price
        perturbed_price = model |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_thm4_array
        simulation_results_task_2_array[i,1] = β_value;
        simulation_results_task_2_array[i,2] = 100*((model.rate - zero_coupon_model.rate)/(zero_coupon_model.rate));    # col 1: percentage in yield
        simulation_results_task_2_array[i,3] = 100*((model.price - VB)/(VB)); # col 2: percentage change in the price of the note
    end
    pretty_table(simulation_results_task_2_array, header=["β","Δr̄ (%)","ΔPrice (%)"] , tf=tf_simple)
end

=========== ========== ==============
         β     Δr̄ (%)    ΔPrice (%) 
=========== ========== ==============
       0.8      -20.0      0.476102
  0.866667   -13.3333      0.316965
  0.933333   -6.66667      0.158331
       1.0        0.0   0.000198586
   1.06667    6.66667     -0.157434
   1.13333    13.3333     -0.314571
       1.2       20.0     -0.471212
=========== ========== ==============


#### Does Theorem 4 hold for T-bills?

In [152]:
does_theorem_4_hold_for_bills = true; # update this value {true | false}

## Task 3: Is price price asymmetry a function of maturity?
Fill me in

In [167]:
short_duration_bond = deepcopy(bond_coupon_model);
short_duration_bond.T = "20-Year" |> securityterm;
short_duration_bond |> discount_model;

Fill me in

In [172]:
let
    VB_short = short_duration_bond.price
    VB_long = bond_coupon_model.price
    simulation_results_task_3_array = Array{Float64,2}(undef, number_of_samples_task_2, 4);
    for i ∈ eachindex(β)
        β_value = β[i]
    
        model_short = deepcopy(short_duration_bond);
        model_long = deepcopy(bond_coupon_model)
        
        ### BEGIN SOLUTION
        model_short.rate = β_value*short_duration_bond.rate
        model_long.rate = β_value*bond_coupon_model.rate
        ### END SOLUTION
        
        # compute: use short-cut syntax and compute the price
        perturbed_price_short = model_short |> discount_model |> x-> x.price
        perturbed_price_long = model_long |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_thm4_array
        simulation_results_task_3_array[i,1] = β_value;
        simulation_results_task_3_array[i,2] = 100*((model_short.rate - short_duration_bond.rate)/(short_duration_bond.rate));    # col 1: percentage in yield
        simulation_results_task_3_array[i,3] = 100*((model_short.price - VB_short)/(VB_short)); # col 2: percentage change in the price of the note
        simulation_results_task_3_array[i,4] = 100*((model_long.price - VB_long)/(VB_long)); # col 2: percentage change in the price of the note
    end
    pretty_table(simulation_results_task_3_array, header=["β","Δr̄ (%)", "ΔPrice-short (%)", "ΔPrice-long (%)"] , tf=tf_simple)
end

=========== ========== ================== ==================
         β     Δr̄ (%)   ΔPrice-short (%)   ΔPrice-long (%) 
=========== ========== ================== ==================
       0.8      -20.0            12.4282           16.0974
  0.866667   -13.3333            8.07545           10.3548
  0.933333   -6.66667            3.93655           4.99867
       1.0        0.0                0.0               0.0
   1.06667    6.66667           -3.74502          -4.66795
   1.13333    13.3333           -7.30872          -9.02979
       1.2       20.0           -10.7007          -13.1082
=========== ========== ================== ==================


#### Is price assymetry ?

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.